<a href="https://colab.research.google.com/github/PRAFULPAWAR8888/Agentic_Ai_Projects-Practice/blob/main/WebChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
from openai import OpenAI
from google.colab import userdata

import os
from PyPDF2 import PdfReader
import gradio as gr


In [153]:
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [154]:

client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta"
)

In [155]:
response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[{"role": "user", "content": "Hello, this is a test!"}]
)
print(response)

ChatCompletion(id='POYnafnbCoyQjrEP2puysQo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello there! I understand this is a test. I'm ready to respond to any questions or prompts you have for me. Let me know what you'd like me to do. Good luck with your test!\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1764222524, model='gemini-2.0-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=45, prompt_tokens=7, total_tokens=52, completion_tokens_details=None, prompt_tokens_details=None))


In [156]:
reader = PdfReader("WebChatbotData/Profile.pdf")
linkedin= ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [157]:
print(linkedin)

   
Contact
At: Bhadawade Post: Shivajinagar
Tal: Khandala Dist: Satara State:
Maharashtra
+91 9322295706  (Mobile)
pawarpraful950@gmail.com
www.linkedin.com/in/praful-
pawar-263526223  (LinkedIn)
github.com/PRAFULPAWAR8888
(Other)
Top Skills
Core Java
MySQL
PHP
Languages
Marathi  (Full Professional)
Hindi  (Full Professional)
English  (Professional Working)
Certifications
Robotics and Artificial Intelligence
Workshop
Career Essentials in Generative AI
by Microsoft and LinkedIn
Career Essentials in Data Analysis
by Microsoft and LinkedIn
Cloud Computing  With
AI( Internship)
DRDO AI /ML ProfestionalPraful Pawar
Artificial Intelligence | Machine learning | Android Development | DSA
| Agentic AI | Data Processing | RAG
Pune, Maharashtra, India
Summary
I am an Artificial Intelligence and Data Science Engineer with a
strong passion for AI, Machine Learning, and Data-Driven problem-
solving. I have hands-on experience in data preprocessing, model
building, fine-tuning, and deploying ML solu

In [158]:
with open("WebChatbotData/summary.txt","r", encoding="utf-8") as f:
    summary = f.read()

In [159]:
name = "Praful Pawar"

In [160]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [161]:
system_prompt

'You are acting as Praful Pawar. You are answering questions on Praful Pawar\'s website, particularly questions related to Praful Pawar\'s career, background, skills and experience. Your responsibility is to represent Praful Pawar for interactions on the website as faithfully as possible. You are given a summary of Praful Pawar\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nPraful Pawar – Summary\n\nI am an Artificial Intelligence and Data Science Engineer with a deep passion for AI, Machine Learning, and intelligent system development. Originally from Maharashtra, India, I thrive on building data-driven solutions that create real-world impact. With strong skills in Python, C++, and Android Development, I specialize in data preprocessing, model training, fine-tuning, and deploying end-to-end M

In [164]:
def chat(message, history):
  messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
  response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=messages)
  return response.choices[0].message.content

In [165]:
gr.ChatInterface(chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0566037224043e8fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
